In [17]:
import pandas as pd

In [18]:
%run helper_functions.ipynb
%run creating_Dataframes.ipynb
%run fixtures.ipynb
%run training_set.ipynb

**get latest GW**

In [19]:
#find latest GW
last_gw = all_season_df['gw'][all_season_df['season'] == '1920'].max()

if np.isnan(last_gw):
    last_gw = 1
else:
    last_gw = last_gw + 1

**get DF for latest GW**

In [20]:
df_latest_gw = build_season(season_paths[3],season_names[3], teams, all_players, [last_gw])

In [21]:
df_latest_gw.drop_duplicates(inplace = True)

**get 'player_proba' for the last GW**

In [22]:
last_gw_player_proba = remaining_season_df[remaining_season_df['gw']== last_gw][['player','play_proba']]

In [23]:
df_latest_gw = df_latest_gw.merge(last_gw_player_proba, on = 'player', how = 'left')

In [24]:
df_latest_gw.drop_duplicates(inplace = True)

**creating training DF**

In [25]:
df_train = pd.concat([all_season_df,df_latest_gw], ignore_index = True, axis = 0,sort = False)

**Next GW**

In [26]:
current_gw = last_gw + 1
fixtures_after_current_gw = pd.read_csv(path/'fixtures.csv')
fixtures_after_current_gw = fixtures_after_current_gw[fixtures_after_current_gw['gw']>= current_gw]

**Merging with 'teams' DF to get team codes in 'gw_fixtures' and keeping relevant columns and renaming the columns**

In [27]:
fixtures_after_current_gw = fixtures_after_current_gw.merge(teams, left_on = 'home_team', right_on = 'team', how ='left')
fixtures_after_current_gw = fixtures_after_current_gw.merge(teams, left_on = 'away_team', right_on = 'team', how ='left')

# keeping relevant columns and renaming the columns
fixtures_after_current_gw = fixtures_after_current_gw[['gw', 'home_team', 'away_team','team_code_x', 'team_code_y']]
fixtures_after_current_gw = fixtures_after_current_gw.rename(index = str,
                                 columns = {'team_code_x':'home_team_code',
                                            'team_code_y':'away_team_code'})

**creating DF for home teams**

In [28]:
#creating DF for home teams
home_df = fixtures_after_current_gw.merge(all_players, left_on = 'home_team_code', right_on = 'team_1920',how = 'left')
#keeping relevant columns
home_df = home_df[['gw','home_team', 'away_team','full_name','position_1920', 'cost_1920', 'play_proba_1920']]
#reanming columns
home_df = home_df.rename(index = str,
                        columns = {'home_team': 'team',
                                  'away_team': 'opponent_team',
                                  'full_name':'player',
                                  'position_1920':'position',
                                  'cost_1920':'price',
                                  'play_proba_1920':'play_proba'})

#setting the flag for 'was_home' to True
home_df['was_home'] =  True

**creating DF for away teams**

In [29]:
#creating DF for home teams
away_df = fixtures_after_current_gw.merge(all_players, left_on = 'away_team_code', right_on = 'team_1920',how = 'left')
#keeping relevant columns
away_df = away_df[['gw','home_team', 'away_team','full_name','position_1920', 'cost_1920', 'play_proba_1920']]
#reanming columns
away_df = away_df.rename(index = str,
                        columns = {'away_team': 'team',
                                   'home_team': 'opponent_team',
                                   'full_name':'player',
                                   'position_1920':'position',
                                   'cost_1920':'price',
                                   'play_proba_1920':'play_proba'})

#setting the flag for 'was_home' to True
away_df['was_home'] =  False

**getting average minutes for the last 5 games**

In [30]:
recent_mins = df_train[(df_train['season'] == '1920') & 
                       (df_train['gw'] >= current_gw - 5)][['player', 'minutes','play_proba']]

#keep only players who were not ruled out due to injury, i.e, play_proba = 1
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

#grouping by players and taking mean
recent_mins_dict = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

**Concatenating home_df and away_df**

In [31]:

# concatenate home and away players
remaining_season_df_after_current_gw = pd.concat([home_df,away_df], ignore_index = True, sort = False, axis =0)

# add season name
remaining_season_df_after_current_gw['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df_after_current_gw['price'] = remaining_season_df_after_current_gw['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df_after_current_gw.loc[remaining_season_df_after_current_gw['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df_after_current_gw['play_proba'] = remaining_season_df_after_current_gw['play_proba'].astype('float') / 100

In [32]:
remaining_season_df_after_current_gw

,gw,team,opponent_team,player,position,price,play_proba,was_home,season
0,31,Burnley,Watford,Matthew_Lowton,2.0,4.2,0.0,True,1920
1,31,Burnley,Watford,Ben_Mee,2.0,5.0,0.0,True,1920
2,31,Burnley,Watford,James_Tarkowski,2.0,5.2,1.0,True,1920
3,31,Burnley,Watford,Ashley_Barnes,4.0,6.1,0.0,True,1920
4,31,Burnley,Watford,Aaron_Lennon,3.0,4.6,0.0,True,1920
...,...,...,...,...,...,...,...,...,...
5235,38,Aston Villa,West Ham United,Marvelous_Nakamba,3.0,4.4,1.0,False,1920
5236,38,Aston Villa,West Ham United,Indiana_Vassilev,3.0,4.5,1.0,False,1920
5237,38,Aston Villa,West Ham United,José_Reina,1.0,4.2,1.0,False,1920
5238,38,Aston Villa,West Ham United,Mbwana_Samatta,4.0,5.2,1.0,False,1920
